In [ ]:
!pip install gdown
import gdown

url = 'https://drive.google.com/file/d/12TpSR-diHPbBWfs-sPisrOPkJ8y_JKXH/view?usp=drive_link'
gdown.download(url, quiet=False,fuzzy=True)

In [ ]:
import zipfile
path_to_zip_file ='/workspaces/Deteksi-Daun-Pisang/dataset.zip'
directory_to_extract_to = '/workspaces/Deteksi-Daun-Pisang/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import os
import numpy as np
import sklearn
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 


data_gambar='/workspaces/deteksi-daun-pisang'
# print(os.listdir(data_gambar))
classes=6
dataset = []
labels = []
img_width=200
img_height=240

for i in range(classes) : 
    path = os.path.join(data_gambar,'dataset/',str(i)) 
    images = os.listdir(path) 
    for a in images : 
        try:  
            image = Image.open(path+'/'+a) 
            image = image.resize((img_width,img_height)) 
            image = np.array(image) 
            print(image) 
            dataset.append(image)  
            labels.append(i) 
        except Exception as e : 
            print(e) 



dataset=np.array(dataset)
labels=np.array(labels)

print(len(dataset))
print(len(labels))
print(dataset.shape,labels.shape)

x_train,x_test,y_train,y_test=train_test_split(dataset,labels,test_size=0.1,random_state=1)

y_train=to_categorical(y_train,6) 
y_test=to_categorical(y_test,6)

model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(img_height,img_width,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6)) 
model.add(Activation('Softmax'))
model.summary() 

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
history=model.fit(x_train,y_train,batch_size=128,verbose=1,epochs=20,validation_data=(x_test,y_test),shuffle=True)
model.save('PenyakitDaunPisang.keras')


test_label=np.argmax(y_test,axis=1)
prediksi=model.predict(x_test)
prediksi=np.argmax(prediksi,axis=-1)

plt.figure(0)
plt.plot(history.history['loss'],label=('Training Loss'))
plt.plot(history.history['val_loss'],label=('Val Loss'))
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

plt.figure(0)
plt.plot(history.history['accuracy'],label=('Training Accuracy'))
plt.plot(history.history['val_accuracy'],label=('Val Accuracy'))
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()


print(classification_report(test_label,prediksi))
print(confusion_matrix(test_label,prediksi))



In [ ]:
import os
import tensorflow as tf
import numpy as np
from flask import *
from keras.preprocessing import image
from PIL import Image
from keras.models import load_model
from sklearn.metrics import accuracy_score
from flask import Flask,render_template
from werkzeug.utils import secure_filename
from sklearn.metrics import accuracy_score


app = Flask(__name__,template_folder='template')


print('Model loaded. Check http://127.0.0.1:5000/')
solusiPenyakitBercakDaunCordana = """
<ol>Solusi:
    <ol>
    Tidak menanam pisang di bawah naungan yang lebat
dan tidak menanam pisang terlalu rapat</li>
    </ol>
"""
solusiPenyakitLayufusarium = """
<ol>Solusi:
    <ol>
    <li>Tidak menanam jenis yang rentan di lahan yang terinfestasi
patogen dan hanya menanam bahan tanaman/anakan yang
sehat.</li>
    <li>Tanaman yang sakit beserta tanah di sekelilingnya dibongkar
dan dikeluarkan dari kebun.
</li>
    <li>Memelihara tanaman dengan hati-hati untuk mengurangi
terjadinya luka-luka pada akar dan mengendalikan nematoda
dengan nematisida.</li>
    </ol>
</ol>
"""
solusiPenyakitburik = """
<ol>Solusi:
    <ol>
    pada umumnya penyakit burik tidak perlu
dikendalikan. Namun jika nanti terasa merugikan, perlu diusahakan
untuk mengurangi peneduhan (karena pohon-pohon) dan penanaman
jangan terlalu padat
    </ol>
</ol>
"""
solusiPenyakitBercakDaunsigatoka = """
<ol>Solusi:
    <ol>
    <li>Pemupukan baik pupuk organic maupun sintetik yang optimal
sehingga membuat tanaman vigor dan relative tan terhadap
penyakit</li>
    <li>Untuk mengurangi sumber infeksi daun-daun mati di
sekeliling pohon dipotong dan dibakar.</li>
    <li>Jika dirasa perlu tanaman dapat disemprot dengan mankozeb
(Dithane M-45) atau propineb (Antracol).</li>
    </ol>
</ol>
"""
solusi = {0:solusiPenyakitBercakDaunCordana,
          1:solusiPenyakitburik,
          2:solusiPenyakitLayufusarium,
          3:solusiPenyakitBercakDaunsigatoka}

classes={0:'<strong>Penyakit Bercak Daun Cordana</strong>'+solusi[0] ,
         1:'<strong>Penyakit Burik</strong>'+solusi[1],
         2:'<strong>Penyakit Layu Fusarium</strong>'+solusi[2],
         3:'<strong>Sehat</strong>',
         4:'<strong>Penyakit Bercak Daun Sigatoka</strong>'+solusi[3],
         5:'<strong>Bukan Daun Pisang</strong>'
        }


def image_processing(img):
    model=load_model('PenyakitDaunPisang.keras')
    data=[]
    image = Image.open(img)
    image = image.resize((200,240))
    data.append(np.array(image))
    image = np.expand_dims(image, axis=0)
    predict=model.predict(image)
    classes_x=np.argmax(predict,axis=1)
    classes_y=np.argmax(predict,axis=-1)
    print("Total Akurasi",accuracy_score(classes_x,classes_y))
    sign = classes[int(classes_x)]
    return sign


@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/predict', methods=['GET','POST'])
def upload():
    if request.method == 'POST':
        f = request.files['file']
        file_path = secure_filename(f.filename)
        f.save(file_path)
        result = image_processing(file_path)
        result = "Ini adalah "+result  
        os.remove(file_path)
        return result
    return None




if __name__ == '__main__':
    app.run(debug=True)